### Importing libraries and downloading packages

In [ ]:
import nltk
import numpy as np 

In [ ]:
# downloading model to tokenize message
nltk.download('punkt')

# downloading stopwords
nltk.download('stopwords')

# downloading wordnet, which contains all lemmas of english language
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import WordNetLemmatizer

### Function to clean text

In [ ]:
def clean_corpus(corpus):
  # lowering every word in text
  corpus = [ doc.lower() for doc in corpus]
  cleaned_corpus = []
  
  stop_words = stopwords.words('english')
  wordnet_lemmatizer = WordNetLemmatizer()

  # iterating over every text
  for doc in corpus:
    # tokenizing text
    tokens = word_tokenize(doc)
    cleaned_sentence = [] 
    for token in tokens: 
      # removing stopwords, and punctuation
      if token not in stop_words and token.isalpha(): 
        # applying lemmatization
        cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token)) 
    cleaned_corpus.append(' '.join(cleaned_sentence))
  return cleaned_corpus

### Loading and cleaning intents

In [ ]:
#

In [ ]:

import json
with open('/content/intents.json') as file:
    intents = json.load(file)


In [ ]:
import json
with open('intents.json', 'r') as file:
  intents = json.load(file)

In [ ]:
corpus = []
tags = []

for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

In [ ]:
cleaned_corpus = clean_corpus(corpus)
cleaned_corpus

['hi',
 'anyone',
 'hey',
 'hola',
 'hello',
 'good day',
 'green tea',
 'lemon tea',
 'would like strong black tea',
 'green tea',
 'tea',
 'would like strong tea',
 'want order tea',
 'want order black tea',
 'much',
 'would like one',
 'much',
 'coffiee',
 'black coffiee',
 'would like strong black coffiee',
 'would like strong coffiee',
 'want order coffiee',
 'want order black coffiee',
 'much',
 'would like one',
 'much',
 'accept cash',
 'accept card',
 '',
 'accept upi',
 '',
 'please check order status',
 'able check order status',
 'order',
 'hey min food',
 'bye',
 'see later',
 'goodbye',
 'nice chatting bye',
 'till next time',
 '',
 'thanks',
 'thank',
 'helpful',
 'awesome thanks',
 'thanks helping',
 'could help',
 'help provide',
 'helpful',
 'support offered',
 'please check order status',
 'able check order status',
 'help order status',
 'order status',
 'order',
 'food',
 'track order',
 'track food',
 'hi want cancel order',
 'want cancel order',
 'please cancel o

### Vectorizing intents

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

### Training neural network

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               7808      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 14)                910       
Total params: 16,974
Trainable params: 16,974
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
70/70 [==============================] - 1s 2ms/step - loss: 2.6378 - accuracy: 0.0571
Epoch 2/20
70/70 [==============================] - 0s 2ms/step - loss: 2.5505 - accuracy: 0.1714
Epoch 3/20
70/70 [==============================] - 0s 2ms/step - loss: 2.4289 - accuracy: 0.3857
Epoch 4/20
70/70 [==============================] - 0s 2ms/step - loss: 2.2667 - accuracy: 0.4571
Epoch 5/20
70/70 [==============================] - 0s 2ms/step - loss: 2.0241 - accuracy: 0.5429
Epoch 6/20
70/70 [==============================] - 0s 1ms/step - loss: 1.6660 - accuracy: 0.7000
Epoch 7/20
70/70 [==============================] - 0s 2ms/step - loss: 1.3492 - accuracy: 0.7857
Epoch 8/20
70/70 [==============================] - 0s 2ms/step - loss: 1.1179 - accuracy: 0.7857
Epoch 9/20
70/70 [==============================] - 0s 2ms/step - loss: 0.9151 - accuracy: 0.8000
Epoch 10/20
70/70 [==============================] - 0s 1ms/step - loss: 0.6876 - accuracy: 0.8429
Epoch 11/20
70/70 [

### Classifying messages to intent

1) If the intent probability does not match with any intent, then send it to no answer.

2) Get Intent

3) Perform Action

In [ ]:
# if prediction for every tag is low, then we want to classify that message as noanswer

INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag

print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag(' chat bot'))
print(predict_intent_tag('Where\'s my order'))

options
payment
order-status-request


In [ ]:
import random
import time 

In [ ]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

In [ ]:
def perform_action(action_code, intent):
  # funition to perform an action which is required by intent
  
  if action_code == 'CHECK_ORDER_STATUS':
    print('\n Checking database \n')
    time.sleep(2)
    order_status = ['in kitchen', 'with delivery executive']
    delivery_time = []
    return {'intent-tag':intent['next-intent-tag'][0],
            'order_status': random.choice(order_status),
            'delivery_time': random.randint(10, 30)}
  
  elif action_code == 'ORDER_CANCEL_CONFIRMATION':
    ch = input('BOT: Do you want to continue (Y/n) ?')
    if ch == 'y' or ch == 'Y':
      choice = 0
    else:
      choice = 1
    return {'intent-tag':intent['next-intent-tag'][choice]}
  
  elif action_code == 'ADD_DELIVERY_INSTRUCTIONS':
    instructions = input('Your Instructions: ')
    return {'intent-tag':intent['next-intent-tag'][0]}

### Generating random responses from the Intents

In [ ]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # check if there's a need to perform some action
  if 'action' in intent.keys():
    action_code = intent['action']
    # perform action
    data = perform_action(action_code, intent)
    # get follow up intent after performing action
    followup_intent = get_intent(data['intent-tag'])
    # generate random response from follow up intent
    response = random.choice(followup_intent['responses'])

    # print randomly selected response
    if len(data.keys()) > 1:
      print('Bot: ', response.format(**data))
    else:
      print('Bot: ', response)

  # break loop if intent was goodbye
  if tag == 'goodbye':
    break 

You: hey
Bot:  Good to see you again
You: do you have tea?
Bot:  yes sir/mam we have that !!!
You: how much for that ?
Bot:  5$ for half cup 
You: do you accept debit/credit card?
Bot:  Yes we accept cash as well as credit/debit cards and we do have upi id as well !
You: how could you help  me?
Bot:  I can check your order status, and cancel your order.
You: check my order status?
Bot:  I am checking your status of your order, Please wait.

 Checking database 

Bot:  Your order is currently with delivery executive, it will be delivered in 21 minutes. Do you need any more help?
You: please check my order status.
Bot:  I am checking your status of your order, Please wait.

 Checking database 

Bot:  Your order is currently in kitchen, it will be delivered in 10 minutes. Do you need any more help?
You: no
Bot:  Yes we accept cash as well as credit/debit cards and we do have upi id as well !
You: yes
Bot:  Yes we accept cash as well as credit/debit cards and we do have upi id as well !
You